# make sure your python version >= 3.10

In [ ]:
!python --version

# Install Yival with pip

In [ ]:
%pip install yival

# Configure your OpenAI API key

In [ ]:
import os
os.environ['OPENAI_API_KEY']= 'XXX'

# Headline Generation demo

This configuration file outlines the setup for an experiment involving the generation of landing page headlines for tech startups. The experiment uses GPT-4, an OpenAI model, and is structured as follows:

## Dataset
The data for the experiment is machine-generated. The prompt  inside instructs GPT-4 to generate a landing page headline for a given tech startup. The data generator will create two examples for each startup.

## Variations
The experiment includes variations of the task, which are also generated by GPT-4. The task is to generate a concise instruction prompt for a given tech startup.

## Evaluators
The experiment uses an individual evaluator type, which assesses the clarity of the generated headline. The evaluator uses a scale from "A" (fails to meet the criterion) to "E" (meets the criterion exceptionally well).

## Selection Strategy
The experiment uses the Analytic Hierarchy Process (AHP) for selection, which considers multiple criteria such as clarity of the headline, average token usage, and average latency. The weights assigned to these criteria are 0.6, 0.2, and 0.2 respectively. The results are normalized using the z-score method.
```
custom_function: demo.headline_generation.headline_generation
description: Generated experiment config
dataset:
  data_generators:
    openai_prompt_data_generator:
      chunk_size: 100000
      diversify: true
      prompt:
          "Please provide a concrete and realistic test case as a dictionary for function invocation using the ** operator.
          Only include parameters, excluding description and name.
          Ensure it's succinct and well-structured.
          **Only provide the dictionary.**"
      input_function:
        description:
          Given an tech startup business, generate a corresponding landing
          page headline
        name: headline_generation_for_business
        parameters:
          tech_startup_business: str
      number_of_examples: 2
      openai_model_name: gpt-4
      output_path: null
  source_type: machine_generated

variations:
  - name: task
    variations:
      - instantiated_value: Generate landing page headline for
        value: Generate landing page headline for
        value_type: str
        variation_id: null
    generator_name: openai_prompt_based_variation_generator
    generator_config:
      openai_model_name: gpt-4
      number_of_variations: 2
      diversify: true
      variables: null
      prompt:
        - content: |-
            Your objective is to construct a concise instruction prompt for GPT-4.
            Task: Given an tech startup business, generate one corresponding landing page headline which is really attracting to all people.
            The name of the company will be add to the end of the prompt.
            keep your output crisp: only the prompt, devoid of any extraneous content.

          role: system

evaluators:
  - evaluator_type: individual
    metric_calculators:
      - method: AVERAGE
    name: openai_prompt_based_evaluator
    display_name: clear
    prompt: |-
      You are assessing a submitted answer on a given task based on a criterion. Here is the data:
      - Task: Given an tech startup business, generate one corresponding landing page headline
      - Does the headline clearly communicate what the startup does or what problem it solves?
        It should be immediately clear to anyone who reads the headline what the startup's purpose is.
        A lack of clarity can lead to confusion and may discourage potential users or investors.
      [Input]: {tech_startup_business}
      [Result]: {raw_output}
      Answer the question by selecting one of the following options:
      A It fails to meet the criterion at all.
      B It somewhat meets the criterion, but there is significant room for improvement.
      C It meets the criterion to a satisfactory degree.
      D It meets the criterion very well.
      E It meets the criterion exceptionally well, with little to no room for improvement.
    choices: ["A", "B", "C", "D", "E"]
    openai_model_name: gpt-4
    description: "evaluate the quality of the landing page headline"
    scale_description: "0-4"
    choice_scores:
      A: 0
      B: 1
      C: 2
      D: 3
      E: 4


selection_strategy:
  ahp_selection:
    criteria:
      - "openai_prompt_based_evaluator: clear"
      - average_token_usage
      - average_latency
    criteria_maximization:
      "openai_prompt_based_evaluator: clear": true
      average_latency: false
      average_token_usage: false
    criteria_weights:
      "openai_prompt_based_evaluator: clear": 0.6
      average_latency: 0.2
      average_token_usage: 0.2
    normalize_func: "z-score"
```

In [ ]:
import os
os.chdir('../..')
!yival run demo/configs/headline_generation.yml